<a href="https://colab.research.google.com/github/yuki-shi/coisinhas-uteis/blob/main/youtube_format_to_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import pandas as pd
import sys
import io
import re

import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

In [ ]:
def format_to_dataframe(response):
  # Listas vazias para os rows e headers formatados
  rows_f = []
  headers_f = []

  response_split = response.split('rows') # separamos a response entre header e rows
  rows = response_split[1].split('[')
  headers = response_split[0].split('name')

  # Extração e formatação do header
  for i in headers[1:]:
    headers_f.append(re.findall(r"(?<=': ).*", i)[0]
                      .split(',')[0])


  headers_f = [x.replace("'", '') for x in headers_f]

  # Extração e formatação do resto
  for row in rows:
    row = row.strip(', ')
    rows_f.append(re.sub(r'["\]\}]', '', row))

  rows_f[-1] = rows_f[-1].strip()

  df = pd.DataFrame([sub.split(',') for sub in rows_f[2:]], columns=headers_f)
  df['day'] = df['day'].str.replace("'", '')

  return df

In [ ]:
old_stdout = sys.stdout
new_stdout = io.StringIO()
sys.stdout = new_stdout


execute_api_request(
    youtubeAnalytics.reports().query,
    ids='channel==MINE',
    startDate='2017-01-01',
    endDate='2017-12-31',
    metrics='estimatedMinutesWatched,views,likes,subscribersGained',
    dimensions='day',
    sort='day'
)


response = new_stdout.getvalue()
sys.stdout = old_stdout

## A solução do stdout foi copiada de https://www.adamsmith.haus/python/answers/how-to-redirect-print-output-to-a-variable-in-python

In [ ]:
format_to_dataframe(response)